# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publicationsold.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [16]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,url_slug,paper_url
0,2024-06-03,A review on microbiological source attribution...,Foodborne pathogens and disease,_Salmonella_ is one of the main causes of huma...,source-att-salmonella-review,https://doi.org/10.1089/fpd.2023.0075
1,2022-06-01,Mathematical modeling of COVID-19 in British C...,Epidemics,Following the emergence of COVID-19 at the end...,age-structured-bc-COVID,https://doi.org/10.1016/j.epidem.2022.100559 ...
2,2021-06-01,How much leeway is there to relax COVID-19 con...,Epidemics,NaN,leeway,http://academicpages.github.io/files/paper1.pdf
3,2021-04-08,Importance of COVID-19 vaccine efficacy in old...,Vaccine,NaN,vaccine,http://academicpages.github.io/files/paper1.pdf
4,2020-12-03,Quantifying the impact of COVID-19 control mea...,PLoS Computational Biology,NaN,paper-bayesian-phys-distancing,http://academicpages.github.io/files/paper2.pdf
5,2020-02-10,Diffusion analysis of single particle trajecto...,Physical Biology,NaN,nonparametrics-bayes-spt,http://academicpages.github.io/files/paper3.pdf
6,2017-09-12,Limitations of Qdot labelling compared to dire...,Scientific reports,NaN,cy3-quantum-dot,http://academicpages.github.io/files/paper3.pdf
7,2014-09-22,Fragmentation of brittle plates by localized i...,Applied Physics Letters,"We address the fragmentation of thin, brittle...",fragmentation-plates,http://academicpages.github.io/files/paper3.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [17]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    #md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    #if len(str(item.paper_url)) > 5:
        #md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
    print(md_filename)   
    with open("../_publications/" + md_filename, 'w') as f:
            f.write(md)

2024-06-03-source-att-salmonella-review.md
2022-06-01 -age-structured-bc-COVID.md
2021-06-01-leeway.md
2021-04-08-vaccine.md
2020-12-03-paper-bayesian-phys-distancing.md
2020-02-10-nonparametrics-bayes-spt.md
2017-09-12-cy3-quantum-dot.md
2014-09-22-fragmentation-plates.md


These files are in the publications directory, one directory below where we're working from.

In [20]:
!ls ../_publications/

2014-09-22-fragmentation-plates.md
2017-09-12-cy3-quantum-dot.md
2020-02-10-nonparametrics-bayes-spt.md
2020-12-03-paper-bayesian-phys-distancing.md
2021-04-08-vaccine.md
2021-06-01-leeway.md
2022-06-01 -age-structured-bc-COVID.md
2024-06-03-source-att-salmonella-review.md


In [21]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
